# 01 - Hardware 

## Operating System

In [92]:
import platform

os_name = platform.system()
print(f"Operating System: {os_name}")

os_details = platform.platform()
print(f"OS Version      : {os_details}")

if os_name=="Linux":
    if "microsoft" in os_details:
        print("--> running inside Windows Subsystem for Linux virtual machine")
    print()
    print("Linux distribution:")
    print()
    !cat /etc/os-release | head -4

Operating System: Linux
OS Version      : Linux-5.15.146.1-microsoft-standard-WSL2-x86_64-with-glibc2.39
--> running inside Windows Subsystem for Linux virtual machine

Linux distribution:

PRETTY_NAME="Ubuntu 24.04 LTS"
NAME="Ubuntu"
VERSION_ID="24.04"
VERSION="24.04 LTS (Noble Numbat)"


## CPU and Memory

In [69]:
import psutil
import platform
import subprocess

# Function to get CPU model name
def get_cpu_model():
    if platform.system() == "Windows":
        command = "wmic cpu get Name"
        cpu_model = subprocess.check_output(command).decode().split("\n")[1].strip()
    elif platform.system() == "Darwin":
        command = "sysctl -n machdep.cpu.brand_string"
        cpu_model = subprocess.check_output(["sysctl", "-n", "machdep.cpu.brand_string"]).decode().strip()
    else:
        command = "cat /proc/cpuinfo | grep 'model name' | uniq"
        cpu_model = subprocess.check_output(command, shell=True).decode().split(":")[1].strip()
    return cpu_model

# Function to get the number of physical cores and logical cores
def get_cpu_core_count():
    physical_cores = psutil.cpu_count(logical=False)
    logical_cores = psutil.cpu_count(logical=True)
    return physical_cores, logical_cores

# Function to get CPU clock speed
def get_cpu_clock_speed():
    cpu_freq = psutil.cpu_freq()
    return cpu_freq.max, cpu_freq.current

# Function to get cache size (Linux specific)
def get_cpu_cache_size():
    cache_size = {}
    try:
        with open('/sys/devices/system/cpu/cpu0/cache/index0/size') as f:
            cache_size['L1'] = f.read().strip()
        with open('/sys/devices/system/cpu/cpu0/cache/index1/size') as f:
            cache_size['L2'] = f.read().strip()
        with open('/sys/devices/system/cpu/cpu0/cache/index2/size') as f:
            cache_size['L3'] = f.read().strip()
    except Exception as e:
        cache_size = {'L1': 'N/A', 'L2': 'N/A', 'L3': 'N/A'}
    return cache_size

# Function to get RAM information
def get_ram_info():
    virtual_memory = psutil.virtual_memory()
    total_ram = virtual_memory.total
    used_ram = virtual_memory.used
    free_ram = virtual_memory.available
    return total_ram, used_ram, free_ram

# Gathering all information
cpu_model = get_cpu_model()
physical_cores, logical_cores = get_cpu_core_count()
max_clock_speed, current_clock_speed = get_cpu_clock_speed()
cache_size = get_cpu_cache_size()
total_ram, used_ram, free_ram = get_ram_info()

# Printing the information
print("----------------------------")
print("CPU")
print("----------------------------")
print(f"CPU Model: {cpu_model}")
print(f"Physical Cores: {physical_cores}")
if logical_cores>physical_cores:
    print(f"Logical  Cores: {logical_cores}")
if max_clock_speed > 0:
    print(f"Max Clock Speed: {max_clock_speed} MHz")
print(f"Current Clock Speed: {current_clock_speed} MHz")
print(f"Cache Size: {cache_size}")
print()
print("----------------------------")
print("Memory")
print("----------------------------")
print(f"Total RAM: {total_ram / (1024 ** 3):.2f} GB")
print(f"Used  RAM: {used_ram / (1024 ** 3):.2f} GB")
print(f"Free  RAM: {free_ram / (1024 ** 3):.2f} GB")

----------------------------
CPU
----------------------------
CPU Model: 13th Gen Intel(R) Core(TM) i7-13700K
Physical Cores: 12
Logical  Cores: 24
Current Clock Speed: 3417.601 MHz
Cache Size: {'L1': '48K', 'L2': '32K', 'L3': '2048K'}

----------------------------
Memory
----------------------------
Total RAM: 31.04 GB
Used  RAM: 1.96 GB
Free  RAM: 28.67 GB


### Monitoring your CPU from a terminal

In [26]:
!top -b -n 1

top - 21:23:50 up  2:44,  0 user,  load average: 0.08, 0.05, 0.01
Tasks:  10 total,   1 running,   9 sleeping,   0 stopped,   0 zombie
%Cpu(s):  0.0 us,  0.0 sy,  0.0 ni, 99.6 id,  0.4 wa,  0.0 hi,  0.0 si,  0.0 st 
MiB Mem :  31785.6 total,  26960.1 free,   2407.2 used,   2841.2 buff/cache     
MiB Swap:   8192.0 total,   8192.0 free,      0.0 used.  29378.5 avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU  %MEM     TIME+ COMMAND
    1 root      20   0    2280   1528   1412 S   0.0   0.0   0:00.01 init(word+
    5 root      20   0    2280      4      0 S   0.0   0.0   0:00.00 init
  722 root      20   0    2284    116      0 S   0.0   0.0   0:00.00 SessionLe+
  723 root      20   0    2300    120      0 S   0.0   0.0   0:00.00 Relay(724)
  724 root      20   0    7872   3516   3156 S   0.0   0.0   0:00.00 bash
  733 root      20   0  571964 123672  20840 S   0.0   0.4   0:08.76 jupyter-l+
  749 root      20   0  768764  83232  17084 S   0.0   0.3   0:00.98 python
  957

### Monitoring your CPU from a notebook

In [67]:
import psutil

# Get CPU usage percentage
cpu_usage = psutil.cpu_percent(interval=1)

# Get memory usage percentage
memory_info = psutil.virtual_memory()

# Display the results
print(f"- CPU load   : {cpu_usage} %")
print(f"- Used memory: {memory_info.used/1024**3:.2f} GB")
print(f"- Free memory: {memory_info.free/1024**3:.2f} GB")

- CPU load   : 0.2 %
- Used memory: 1.95 GB
- Free memory: 26.31 GB


## Nvidia GPU

NVLM API reference: https://docs.nvidia.com/deploy/nvml-api/index.html

In [59]:
from pynvml import *

nvmlInit()

deviceCount = nvmlDeviceGetCount()
for i in range(deviceCount):
    print("----------------------------")
    print("GPU")
    print("----------------------------")
    handle = nvmlDeviceGetHandleByIndex(i)
    # Fix bug: https://github.com/gpuopenanalytics/pynvml/issues/53
    if nvmlSystemGetDriverVersion() != "555.99": 
        print("Device", i, ":", nvmlDeviceGetName(handle))
    else:
        arch_map = { 1: "Kepler", 2: "Maxwell", 3: "Pascal", 4: "Volta", 5: "Turing", 6: "Ampere", 7: "Ada", 8: "Hopper", 0: "Unknown" }
        print("Device", i, ": architecture", arch_map[nvmlDeviceGetArchitecture(handle)])      
    print("- Total memory  :",int(info.total/1024/1024), "MB")    
    print("- Driver Version:", nvmlSystemGetDriverVersion())
    cuda_driver_version = nvmlSystemGetCudaDriverVersion()
    print(f"- Driver CUDA   : {cuda_driver_version//1000}.{(cuda_driver_version%1000)//10}")
    print()

----------------------------
GPU
----------------------------
Device 0 : architecture Hopper
- Total memory  : 24564 MB
- Driver Version: 555.99
- Driver CUDA   : 12.5



### Monitoring your GPU from Terminal 

In [1]:
!nvidia-smi

Sat Jun  8 19:33:29 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.52.01              Driver Version: 555.99         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:01:00.0  On |                  Off |
| 31%   32C    P8              9W /  450W |     822MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Monitoring your GPU from a notebook

In [63]:
deviceCount = nvmlDeviceGetCount()
for i in range(deviceCount):
    handle = nvmlDeviceGetHandleByIndex(i)
    print("GPU", i)

    utilization = nvmlDeviceGetUtilizationRates(handle)
    gpu_utilization = utilization.gpu
    print(f"- GPU load   : {gpu_utilization}%")
    print(f"- Power usage: {nvmlDeviceGetPowerUsage(handle)//1000} W")
    
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"- Used memory: {int(info.used/1024/1024)} MB")
    print(f"- Free memory: {int(info.free/1024/1024)} MB")

GPU 0
- GPU load   : 0%
- Power usage: 10 W
- Used memory: 1280 MB
- Free memory: 23283 MB
